**Imports**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### Data Reading

In [0]:
products = spark.read.format('parquet').load('abfss://bronze@endtoenddatabricks.dfs.core.windows.net/products')

# Removing the unwanted columns
products = products.drop('_rescued_data')
products.display()

product_id,product_name,category,brand,price
P0001,Clearly Its,Beauty,Nike,1868.54
P0002,Production Clear,Beauty,Apple,587.13
P0003,Culture Coach,Home,Revlon,1599.24
P0004,Movement Part,Sports,LG,651.71
P0005,Fact Name,Clothing,Samsung,1861.78
P0006,Usually Stop,Toys,Adidas,936.36
P0007,Reveal Current,Sports,Adidas,1954.02
P0008,Force Language,Beauty,Puma,1251.26
P0009,Stage Leg,Clothing,Samsung,1247.15
P0010,Leader Then,Sports,Sony,975.53


**products available per brand**

In [0]:
products.groupBy('brand').agg(count(col('product_id')).alias('products per brand')).sort(col('products per brand').desc()).display()

brand,products per brand
Adidas,61
Samsung,60
Lenovo,55
Sony,50
Nike,49
Puma,48
Apple,48
Dell,45
Revlon,42
LG,42


### Functions

In [0]:
products.createOrReplaceTempView('v_products')

### Creating functions using SQL

In [0]:
%sql
CREATE OR REPLACE FUNCTION `catalog-databricks`.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
SELECT product_id, price, round(`catalog-databricks`.bronze.discount_func(price), 2) discounted_price 
FROM v_products;

product_id,price,discounted_price
P0001,1868.54,1681.69
P0002,587.13,528.42
P0003,1599.24,1439.32
P0004,651.71,586.54
P0005,1861.78,1675.6
P0006,936.36,842.72
P0007,1954.02,1758.62
P0008,1251.26,1126.13
P0009,1247.15,1122.44
P0010,975.53,877.98


### Creating functions using Python

In [0]:
%sql
CREATE OR REPLACE FUNCTION `catalog-databricks`.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS 
$$
  return p_brand.upper()
$$

In [0]:
products.limit(3).display()

product_id,product_name,category,brand,price
P0001,Clearly Its,Beauty,Nike,1868.54
P0002,Production Clear,Beauty,Apple,587.13
P0003,Culture Coach,Home,Revlon,1599.24


In [0]:
%sql
SELECT product_name, `catalog-databricks`.bronze.upper_func(product_name) upper_prd_nm, brand, `catalog-databricks`.bronze.upper_func(brand) upper_brand_nm
FROM v_products

product_name,upper_prd_nm,brand,upper_brand_nm
Clearly Its,CLEARLY ITS,Nike,NIKE
Production Clear,PRODUCTION CLEAR,Apple,APPLE
Culture Coach,CULTURE COACH,Revlon,REVLON
Movement Part,MOVEMENT PART,LG,LG
Fact Name,FACT NAME,Samsung,SAMSUNG
Usually Stop,USUALLY STOP,Adidas,ADIDAS
Reveal Current,REVEAL CURRENT,Adidas,ADIDAS
Force Language,FORCE LANGUAGE,Puma,PUMA
Stage Leg,STAGE LEG,Samsung,SAMSUNG
Leader Then,LEADER THEN,Sony,SONY


In [0]:
products.select(expr("`catalog-databricks`.bronze.upper_func(product_name)").alias("uppercase_prod_nms"))

DataFrame[uppercase_prod_nms: string]

### Writing the cleaned data

In [0]:
products.write.format('delta').mode('overwrite').save('abfss://silver@endtoenddatabricks.dfs.core.windows.net/products')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS `catalog-databricks`.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@endtoenddatabricks.dfs.core.windows.net/products'